# 프로젝트: Movielens 영화 추천 

### 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다. 별점을 시청횟수로 해석해서 생각하겠습니다. 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

1) 데이터 준비와 전처리

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'like'}, inplace=True)

In [4]:
ratings['like']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: like, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
#movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = "ISO-8859-1")
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# 검색하기 쉽게 하기위해 타이틀 문자열을 소문자로 변경
movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


#### 여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.



### 2) 분석해 봅시다.

### ratings에 있는 유니크한 영화 개수

In [7]:
ratings['movie_id'].nunique()

3628

### rating에 있는 유니크한 사용자 수

In [8]:
ratings['user_id'].nunique()

6039

### 가장 인기 있는 영화 30개(인기순)

In [9]:
# 인기 많은 영화 30개  
movie_count_rating = ratings.groupby('movie_id')['like'].count()
movie_count_rating.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: like, dtype: int64

3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다

In [10]:
data = ratings[['user_id','movie_id','like']]

In [11]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['toystroy' , 'avengers' ,'joker' ,'jumanji' ,'godzilla']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zoo']*5, 'movie_id': my_favorite, 'like':[5]*5})

if not data.isin({'user_id':['zoo']})['user_id'].any():  # user_id에 'zoo'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,like
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,zoo,toystroy,5
1,zoo,avengers,5
2,zoo,joker,5
3,zoo,jumanji,5
4,zoo,godzilla,5


In [12]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['movie_id'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [13]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zoo'])     
print(movie_to_idx['toystroy'])

6039
3628


In [14]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

data

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,like
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,3628,5
1,6039,3629,5
2,6039,3630,5
3,6039,3631,5


4) CSR matrix를 직접 만들어 봅시다.

In [15]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()

csr_data = csr_matrix((data.like, (data.user_id, data.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [17]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.02, iterations=15, dtype=np.float32)

In [18]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [19]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [20]:
zoo, avengers = user_to_idx['zoo'], movie_to_idx['avengers']
zoo_vector, avengers_vector = als_model.user_factors[zoo], als_model.item_factors[avengers]

print('슝=3')

슝=3


In [21]:
zoo_vector

array([-1.0076932e-03,  2.2995979e-05, -5.6364539e-04, -1.1632685e-03,
        1.2470398e-03,  3.9634495e-03, -2.5864858e-03, -2.5833265e-03,
       -1.2568159e-03,  4.2110858e-03,  1.2167776e-03,  1.3473075e-03,
        2.1238369e-03,  2.4111057e-04,  2.5735311e-03,  2.9311182e-03,
        6.0075475e-04,  1.2694903e-03, -1.7250025e-03, -1.1755061e-05,
        6.8680653e-03,  3.9158459e-03,  3.2552767e-03, -2.3822745e-03,
        1.8323850e-03, -2.3179809e-03,  2.2976394e-03,  4.3222567e-04,
        4.9729459e-03, -1.6819795e-03, -1.5691444e-03, -2.7188060e-03,
        3.4715394e-03, -2.2658315e-03,  1.4359327e-03,  2.0698602e-03,
       -2.8808690e-03, -9.7954238e-04,  2.1228450e-03, -5.9101726e-03,
        2.1615224e-03, -2.0146661e-03,  2.2137507e-03, -1.9090899e-03,
       -4.6308776e-03,  1.9778039e-03, -3.7384281e-04,  9.5035508e-04,
        3.9042367e-04,  7.4641040e-04,  4.1915239e-03,  2.6894191e-03,
       -4.9879780e-04,  2.9169045e-05, -1.0486981e-03, -2.5147002e-03,
      

In [22]:
avengers_vector

array([ 5.68228861e-05,  3.48900830e-05,  2.64230334e-06,  5.79749649e-05,
        9.58477758e-05,  1.49634230e-04,  1.89035800e-05,  2.68122421e-05,
        6.49227513e-05,  1.48184787e-04,  1.12305745e-04,  5.86331589e-05,
        1.39144628e-04,  4.17425690e-05,  1.03602564e-04,  1.39919095e-04,
        9.30753959e-05,  6.78567012e-05,  6.25272369e-05,  5.72301760e-05,
        2.60021770e-04,  1.60737894e-04,  1.22238984e-04,  4.63873512e-05,
        1.02277372e-04,  2.78432508e-05,  1.40326665e-04,  1.18188946e-04,
        2.09326667e-04, -1.36631052e-05,  3.77684082e-05, -1.27438334e-05,
        1.65488746e-04, -1.22568611e-06,  1.63772842e-04,  1.07814602e-04,
       -2.39920009e-05,  2.91012966e-05,  1.54070745e-04, -6.92708927e-05,
        1.10963018e-04,  4.61150266e-05,  1.33040667e-04,  1.25691640e-05,
       -8.15873937e-05,  1.66394719e-04,  6.13902012e-05,  7.85892335e-05,
        4.07177977e-05,  7.75793669e-05,  1.77556751e-04,  1.90984298e-04,
        8.40846333e-05,  

In [23]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zoo_vector, avengers_vector)

1.7487142e-05

In [24]:
avengers = movie_to_idx['avengers']
avengers_vector = als_model.item_factors[avengers]
np.dot(zoo_vector, avengers_vector)

1.7487142e-05

7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [25]:
favorite_movie = 'avengers'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3629, 0.9999999),
 (3631, 0.9949312),
 (3632, 0.992222),
 (3628, 0.9917565),
 (3630, 0.9827167),
 (3578, 0.9314408),
 (3583, 0.9309889),
 (3572, 0.92978525),
 (3579, 0.9290853),
 (3580, 0.928639),
 (3576, 0.9284962),
 (3573, 0.9283162),
 (3575, 0.928213),
 (3577, 0.92817205),
 (3574, 0.92785954)]

In [27]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['avengers',
 'jumanji',
 'godzilla',
 'toystroy',
 'joker',
 2214,
 138,
 584,
 763,
 3229,
 1548,
 2821,
 1510,
 701,
 607]

In [30]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [31]:
get_similar_movie('avengers')

['avengers',
 'jumanji',
 'godzilla',
 'toystroy',
 'joker',
 2214,
 138,
 584,
 763,
 3229]

8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [32]:
user = user_to_idx['zoo']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(953, 0.0012292963),
 (41, 0.0011953434),
 (1837, 0.0010787196),
 (2661, 0.0010567146),
 (2106, 0.000996625),
 (834, 0.0009825903),
 (2376, 0.00097607693),
 (706, 0.00097356807),
 (232, 0.0009531665),
 (224, 0.000951102),
 (613, 0.0009365728),
 (110, 0.0009341459),
 (2044, 0.0009112835),
 (2609, 0.0009052575),
 (476, 0.0008760444),
 (488, 0.00085354026),
 (2095, 0.0008508279),
 (155, 0.00084481225),
 (107, 0.0008427473),
 (2229, 0.00083196594)]

In [33]:
[idx_to_movie[i[0]] for i in movie_recommended]

[3396,
 1961,
 3185,
 635,
 3936,
 2961,
 2621,
 2108,
 1046,
 1617,
 357,
 1265,
 2924,
 725,
 1148,
 1307,
 3384,
 2943,
 480,
 1415]

In [34]:
rihanna = movie_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

KeyError: 'rihanna'

In [35]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

NameError: name 'explain' is not defined

# 프로젝트를 마치며... 